In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

data_calendar = pd.read_csv('../Data/calendar.csv')

In [8]:
data_calendar.loc[data_calendar['listing_id'] == 2595, 'available'].value_counts()

t    340
f     26
Name: available, dtype: int64

In [2]:
na_column = data_calendar.apply(lambda x: sum(x.isnull()), axis = 0)

def num_na(x):
    return sum(x.isnull())
na_by_id = (data_calendar
     .groupby('listing_id')
     .agg(
        num_price_na = ('price', num_na),
        num_adj_price_na = ('adjusted_price', num_na),
        num_min_night_na = ('minimum_nights', num_na),
        num_max_night_na = ('maximum_nights', num_na)
     ))

has_na_listing_id = na_by_id[na_by_id.apply(lambda x: any(x.iloc[0:4]), axis = 1)]

In [3]:
na_by_id.head()

,num_price_na,num_adj_price_na,num_min_night_na,num_max_night_na
listing_id,,,,
2595,0,0,0.0,0.0
3831,0,0,1.0,1.0
5099,0,0,0.0,0.0
5121,0,0,0.0,0.0
5178,0,0,0.0,0.0


In [4]:
has_na_listing_id[has_na_listing_id['num_max_night_na'] > 0].head()

,num_price_na,num_adj_price_na,num_min_night_na,num_max_night_na
listing_id,,,,
3831,0,0,1.0,1.0
14377,0,0,1.0,1.0
21293,0,0,1.0,1.0
44161,0,0,1.0,1.0
75635,0,0,1.0,1.0


In [5]:
price_na_listing_id = na_by_id[(na_by_id['num_price_na'] > 0) | (na_by_id['num_adj_price_na'] > 0)].index
# five listings have missing data, the number of rows with missing data is around 170, almost half the 365 days,
# therefore, we delete the record of this 5 listings
len(price_na_listing_id)
data_calendar = data_calendar[~data_calendar['listing_id'].isin(price_na_listing_id)]

In [6]:
sum(data_calendar['price'].isnull())

0

In [7]:
def to_num(x):
    x = re.sub(r',', '', x)
    return float(x[1:])
data_calendar['price'] = list(map(to_num, data_calendar['price']))
data_calendar['adjusted_price'] = list(map(to_num, data_calendar['adjusted_price']))

In [8]:
calendar_summary = (
        data_calendar
        .groupby('listing_id')
        .agg({
            'available': lambda x: sum(x == 't') / len(x),
            'price': np.mean,
            'adjusted_price': np.mean,
            'minimum_nights': np.mean,
            'maximum_nights': np.mean
        })
)

In [14]:
calendar_summary.rename(columns = {'price':'price_avrg', 'adjusted_price':'adjusted_price_avrg'}, inplace=True)
calendar_summary.head()

,available,price_avrg,adjusted_price_avrg,minimum_nights,maximum_nights
listing_id,,,,,
2595,0.928962,196.415301,196.415301,10.000000,1125.0
3831,0.445355,86.221311,86.221311,1.000000,730.0
5099,0.052055,206.164384,206.164384,3.000000,21.0
5121,1.000000,60.000000,60.000000,29.000000,730.0
5178,0.663014,78.279452,78.279452,1.750685,14.0


In [15]:
calendar_summary.to_csv("calendar_summary.csv")

In [38]:
sum(calendar_summary['maximum_nights'].isnull())

0

In [6]:
a = data_calendar[data_calendar['listing_id'] == 2595]

In [10]:
np.mean(a['adjusted_price'])

196.4153005464481

In [23]:
calendar_summary.shape

(50594, 5)

In [4]:
np.mean([np.nan, 1, 3])

nan